In [5]:
import pandas as pd
import torch
import os
import pathlib
import sys
sys.path.append("../")

from unet.utils.data_utils import generate_patches, save_patches

In [ ]:
# Make a patch dataset
# Basically, the ground truth is large 3D images (24, 1024, 1024) (z, y, x), so instead of generating patches on the fly
# before feeding to the model, this will allow for batching of different patches to be passed and hopefully 
# better model performance.

load_data = pd.read_csv("./data/data.csv")

patch_shape = (24, 200, 200)
stride_shape = (24, 100, 100)

save_df = {
    "image": [],
    "label": []
    }

for im, ma in zip(load_data.iloc[:, 0].values, load_data.iloc[:, 2].values):
    image = torch.from_numpy(skimage.io.imread(im).astype(np.float32))
    image_filename = os.path.basename(im)
    image_patches = generate_patches(image, patch_shape=patch_shape, stride_shape=stride_shape)
    image_save_paths = save_patches(image_patches, image_filename, "./patch_data/image")

    save_df["image"].extend(image_save_paths)

    mask = torch.from_numpy(skimage.io.imread(ma).astype(np.float32))
    mask_filename = os.path.basename(ma)
    mask_patches = generate_patches(mask, patch_shape=patch_shape, stride_shape=stride_shape)
    mask_save_paths = save_patches(mask_patches, mask_filename, "./patch_data/labels")

    save_df["label"].extend(mask_save_paths)

save_df = pd.DataFrame.from_dict(save_df, orient="index").transpose()
save_df.to_csv("./patch_data/load_data.csv", index=False)

In [14]:
# Make a new DF that excludes acquisition 1, which will be used for testing

df = pd.read_csv("../patch_data/load_data.csv")

df = df[~df["image"].str.contains("acqusition-1")]

df.to_csv("../patch_data/load_data_training.csv", index=False)